## Projektsökvägar

In [1]:
from pathlib import Path
import os
def paths():    
    prj_dir = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29-test")
    LGs_info_path = os.path.join(prj_dir, 'Ledningar_Kantträd_2023-05-29.txt')
    gdb = os.path.join(prj_dir, 'SVK_SKB_2023-04-27.gdb')
    return prj_dir, gdb, LGs_info_path

In [2]:
prj_dir = paths()[0]
LGs_info_name = 'Ledningar_Kantträd_2023-05-29.txt'
LGs_info_path = os.path.join(prj_dir, LGs_info_name)
txt_file = 'Ledningar_Kantträd_2023-05-29.txt'
gdb_name = 'Kantträd_SKB_2023-05-29.gdb'
gdb = os.path.join(prj_dir, gdb_name)
serv_prj_dir = Path(r"Z:\Projekt\Analys\ledningar")
cvd_LEDNINGSGATA = 'cvd_LEDNINGSGATA.txt'
cvd_LEDNINGSGATA_path = os.path.join(prj_dir, cvd_LEDNINGSGATA)
cvd_DATE = 'cvd_DATE.txt'
cvd_DATE_path = os.path.join(prj_dir, cvd_DATE)
akuta_merged_name = "akuta_2023_05_29"
ej_akuta_merged_name = "ej_akuta_2023_05_29"
version = "2023_05_29"

In [3]:
serv_prj_dir

WindowsPath('Z:/Projekt/Analys/ledningar')

# Merge SKB text files for all blocks of a powerline

In [3]:
import os
import sys
import glob
import fileinput
from pathlib import Path
sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata")
import SVK
import pandas as pd

#LGs_info_path = os.path.join(prj_dir, txt_file)
#LGs_info_path = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\Ledningar_Kantträd_2023-05-29.txt"

#serv_prj_dir = Path(r"Z:\Projekt\Analys\ledningar")

def combine_blocks(row):
    LG = row[0]
    line = row[1]
    line_dir = serv_prj_dir / LG / f"line_{line}"
    
    print(f"Doing {LG}_{line}")
    #block_dir = line_dir / "Kantträd" / "block"
    block_dir = os.path.join(line_dir, "kantträd", "block") #'Ledningar_2022_Kantträd_2023-02-06.txt')
    #combined_blocks_path = line_dir / "Kantträd" / f"SKB_raw.txt"
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    
    # Merge RBX files for all blocks into one
    SVK.merge_blocks(block_dir, "*_pt*.txt", combined_blocks_path) # DENNA SKILDE SIG FRÅN 2022 till 2021. Tidigare "*SKB_"

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
print(LGs_info)
LGs_info.apply(combine_blocks, axis=1)

      LG  line   Littera                Namn  Strömtyp  Spänning  LG_kod  \
0  LG010     3  UL6 S5-7  Trolltjärn-Betåsen        10       400     100   

      Status  
0  Befintlig  
Doing LG010_3


0    None
dtype: object

## Kör CopySKBfiles.py
### Denna kopierar över alla skb_raw.txt till den temporära mappen på skrivbordet

# Calculate distances to wire for trees

In [4]:
# Prepare SKB feature class (point) with X,Y,Z,dZ,mZ,avt_fas,avst_hori
# (dZ = tree height, mZ = ground elevation at tree base)
# Point coordinates are X,Y,mZ
import sys
#sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata\kantträd")
import os, SKB, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd

# UPPDRAGSMAPP
#prj_dir = paths()[0]
#prj_dir = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04")
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
#LGs_info_path = os.path.join(prj_dir, 'Ledningar_Kantträd_2023-05-29.txt')

# LOKALT
#gdb = os.path.join(prj_dir, gdb_name)
#gdb = Path(r"C:\Users\semnve\Desktop\Exempeldata_2021\SVK_2021_GIS\SVK_2021_GIS.gdb")

SKB_XYmZ_location = os.path.join(gdb, 'SKB_XYmZ')
SKB_XYZ_location = os.path.join(gdb, 'SKB_XYZ')
#input_dir = os.path.join(prj_dir, 'SKB_alla_LG')
#SKB_file_prefix = 'SKB_'
wires_fd_1 = os.path.join(gdb, 'Wires')
wires_fd_2 = os.path.join(gdb, 'Wires_incomplete')
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(gdb, 'SKB_template')
atgardsar = 2024
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row[0]
    line = row[1]
    line_dir = prj_dir / "ledningar" / LG / f"line_{line}"
    SKB_dir = line_dir / "SKB"
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")

    #SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    #SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    #akuta_trad = f"{LG}_{line}_akuta"
    #ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad)
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad)

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}")
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    elif arcpy.Exists(wires_2):
        wires = wires_2
        print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    #SKB_file_in = os.path.join(SKB_dir, f"{SKB_file_prefix}{LG}_{line}.txt")
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    print(f'{LG}_{line}_SKB_XYmZ created')

    # Call arcpy function Near3D to calculate distance from mZ to wire, 
    # on SKB_XYmZ
    SKB.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    print(f'dist_mZ_wire calculated for {LG}_{line}')
    print(f'{strftime("%H:%M:%S", gmtime())}')

    # Create feature class for SKB where point XYZ is the XYZ for the tree tops
    SKB.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)

    # Fill SKB_XYZ with values from SKB_XYmZ and set Z coordinate to Z (instead of mZ)
    LG_nr = int(LG[-3:])
    SKB.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod,
                        matosakerhet_p, matosakerhet_h)
    print('SKB_XYZ populated')  

    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the 
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    SKB.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    print('Avstand fas computed')
          
    # Now replace old avst_hori with new, correct one from compute_horizontal_dist. 
    # Then remove the NEAR-fields.
    SKB.update_horizontal_dist(SKB_XYZ_path)
    print('Avstand horisontellt computed')
          
    # Pick out akuta trad (avstand fas <= 1)
    SKB.pick_out_akuta(SKB_XYZ_location, SKB_XYZ, akuta_trad)
    print('Akuta exported')
          
    # Remove duplicates from akuta_trad
    arcpy.DeleteIdentical_management(akuta_trad_path, ['Shape'], 0.05, 0.05)
    print('Duplicates removed')
    
    # Pick out ej akuta trad (avstand fas > 1)
    SKB.pick_out_ej_akuta(SKB_XYZ_location, SKB_XYZ, ej_akuta_trad)
    print('ej_akuta_trad exported')
    
    # Remove duplicates from ej_akuta_trad
    arcpy.DeleteIdentical_management(ej_akuta_trad_path, ['Shape'], 0.05, 0.05)
    print('Duplicates removed')
     
    print("Setting ground Z for XYZ points")
    ej_akuta_fc = os.path.join(SKB_XYZ_path, ej_akuta_trad)
    akuta_fc = os.path.join(SKB_XYZ_path, akuta_trad)
    all_fc = os.path.join(SKB_XYZ_path, SKB_XYZ)
    SKB.update_z_coordinate(SKB_XYZ_location, SKB_XYZ)
    SKB.update_z_coordinate(SKB_XYZ_location, akuta_trad)
    SKB.update_z_coordinate(SKB_XYZ_location, ej_akuta_trad)
    
          
    print(f"Done with {LG}_{line}")
# Coordinate system
sr = sweref99TM()

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(compute_SKB_distances, axis=1)
    


Starting with LG010_3
LG010_3 faslinor i Wires
LG010_3_SKB_XYmZ created
done with Near3D LG010_3_SKB_XYmZ
done with deletefield LG010_3_SKB_XYmZ
dist_mZ_wire calculated for LG010_3
13:38:42
SKB_XYZ populated
Avstand fas computed
Avstand horisontellt computed
Akuta exported
Duplicates removed
ej_akuta_trad exported
Duplicates removed
Setting ground Z for XYZ points
Done with LG010_3


0    None
dtype: object

## Scan dates and accuracies to Akuta

In [5]:
#ANVÄNDS

import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

SKB_XYZ_fd = os.path.join(prj_dir, gdb ,"SKB_XYZ")                                                                                            

def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    for fieldname in fieldnames:
        
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        print(f"Altering fields for {LG}_{line}{fieldname}")
        path_joined = os.path.join(SKB_XYZ_fd, f"{LG}_{line}{fieldname}")
        field_names = [f.name for f in arcpy.ListFields(path_joined)]
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if "LEDNINGSGATA" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "LEDNINGSGATA")
        if "INSAMLINGSMETOD" in field_names:    
            arcpy.management.DeleteField(
                path_joined,
                "INSAMLINGSMETOD")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG010_3_SKB_XYZ
Altering fields for LG010_3_akuta
Altering fields for LG010_3_ej_akuta


0    None
dtype: object

In [6]:
# Add data from LEDNIGNAR.txt to the akuta
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

#gdb = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\Kantträd_SKB_2023-05-29.gdb")
#LGs_info_file = "Ledningar_Kantträd_2023-05-29.txt"
#prj_dir = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29"

#LGs_info_path = os.path.join(prj_dir, LGs_info_file)
#cvd_LEDNINGSGATA_path = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\cvd_LEDNINGSGATA.txt"
#cvd_DATE_path = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\cvd_DATE.txt"

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    print(line)
    #if len(str(line)) == 1:
    #    LG_date = cvd_DATE[LG.join("_00" + str(line))]
    #if len(str(line)) == 2:
    #    LG_date = cvd_DATE[LG.join("_0" + str(line))]
    #if len(str(line)) == 3:
    #    LG_date = cvd_DATE[LG.join("_" + str(line))]

    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
    len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join(
    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)

3
Klart


0    None
dtype: object

# Adderar fält
### Adderar fält till feature classes och importerar information från lednignar.txt

In [7]:
# Add data from LEDNIGNAR.txt to the akuta
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

#gdb = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\Kantträd_SKB_2023-05-29.gdb")
#LGs_info_file = "Ledningar_Kantträd_2023-05-29.txt"
#prj_dir = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29"

#LGs_info_path = os.path.join(prj_dir, LGs_info_file)
#cvd_LEDNINGSGATA_path = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\cvd_LEDNINGSGATA.txt"
#cvd_DATE_path = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\cvd_DATE.txt"

SKB_location = os.path.join(gdb, 'SKB_XYZ')

date_field = 'RGDTM'
matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline[0]
    line = powerline[1]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    #if len(str(line)) == 1:
    LG_date = cvd_DATE[LG.join("_00" + str(line))]
    #if len(str(line)) == 2:
    #    LG_date = cvd_DATE[LG.join("_0" + str(line))]
    #if len(str(line)) == 3:
    #    LG_date = cvd_DATE[LG.join("_" + str(line))]
    
    #SKB_path = os.path.join(SKB_location, f"{LG}_{line}_SKB_XYZ")

    #fc_SKB = f"{LG}_{line}_SKB_XYZ"
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")

    #if arcpy.Exists(SKB_path):
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True

        # Creating actual fields in the geodatabase and calculating values
        arcpy.management.AddField(fc_SKB, "LITTERA", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "LITTERA", "".join(("'", littera, "'")))
        arcpy.management.AddField(fc_SKB, "LEDNINGSGATA", "LONG")
        arcpy.management.CalculateField(fc_SKB, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(fc_SKB, "RGDTM", "DATE")
        arcpy.management.CalculateField(fc_SKB, "RGDTM", "".join(("'", LG_date, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung", "TEXT")
        arcpy.management.CalculateField(fc_SKB, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(fc_SKB, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(fc_SKB, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(fc_SKB, "Insamlingsmetod", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Plan", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(fc_SKB, "Matosakerhet_Hojd", "LONG")
        arcpy.management.CalculateField(fc_SKB, "Matosakerhet_Hojd", int(matosak_hojd))
        #field = ['AVSTAND_FAS']
        #arcpy.management.CalculateField(fc_SKB, field, round(field[0], 1))
        #arcpy.CalculateField_management(SKB_path, "AVSTAND_FAS", '"{:.1f}".format(round(!AVSTAND_FAS!, 1))', "PYTHON_9.3")

        dataset = SKB_location
        sr = arcpy.SpatialReference("SWEREF99_TM")
        arcpy.DefineProjection_management(dataset, sr)
        # Assigning domains to fields
        arcpy.management.AssignDomainToField(
            fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(
            fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(
            fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")

    print("Klart")

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
    len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {df_cvd_DATE.LG[i].join(
    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

LGs_info.apply(add_date_and_accuracy, axis=1)


Doing LG010_3
Klart


0    None
dtype: object

In [8]:
import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

#LGs_info_file = "Ledningar_Kantträd_2023-05-29.txt"
#prj_dir = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29"
#SKB_XYZ_fd = os.path.join(prj_dir,"Kantträd_SKB_2023-05-29.gdb","SKB_XYZ")                                                                                            

#LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_location, f"{LG}_{line}{fieldname}")
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]
            if "AVSTAND_HORISONTELLT" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT",
                    "AVSTAND_HORISONTELLT")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_HORISONTELLT']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "AVSTAND_FAS" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "AVSTAND_FAS",
                    "AVSTAND_FAS",
                    "AVSTAND_FAS")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['AVSTAND_FAS']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "DELTA_HOJD" in field_names:    
                arcpy.management.AlterField(
                    path_joined,
                    "DELTA_HOJD",
                    "DELTA_HOJD",
                    "DELTA_HOJD")

                arcpy.env.workspace = SKB_XYZ_fd
                field = ['DELTA_HOJD']
                cursor = arcpy.UpdateCursor(SKB_XYZ_fc,field)

                with arcpy.da.UpdateCursor(SKB_XYZ_fc,field) as cursor:
                    for row in cursor:
                        row[0] = float("{:.1f}".format(round(row[0], 1)))
                        cursor.updateRow(row)

            if "RGDTM" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "RGDTM",
                    "RGDTM",
                    "RGDTM")

            if "MATOSAKERHET_PLAN" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_PLAN",
                    "Matosakerhet_Plan",
                    "Matosakerhet_Plan")

            if "MATOSAKERHET_HOJD" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "MATOSAKERHET_HOJD",
                    "Matosakerhet_Hojd",
                    "Matosakerhet_Hojd")

            if "LEDNINGSGATA" in field_names: 
                arcpy.management.AlterField(
                    path_joined,
                    "LEDNINGSGATA",
                    "LEDNINGSGATA",
                    "LEDNINGSGATA")

            #if "GIS_ID" in field_names:    
            #        arcpy.management.DeleteField(
            #            path_joined,
            #            "GIS_ID")
            #if "MAX_TILLVAXT" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "MAX_TILLVAXT")
            #if "created_user" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "created_user")
            #if "created_date" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "created_date")
            #if "last_edited_user" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "last_edited_user")
            #if "last_edited_date" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "last_edited_date")
            #if "ATGARDSAR" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "ATGARDSAR")
            #if "LEVERANTOR" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "LEVERANTOR")
            #if "LEVERANTORS_DATUM" in field_names:    
            #    arcpy.management.DeleteField(
            #        path_joined,
            #        "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG010_3_SKB_XYZ
Altering fields for LG010_3_akuta
Altering fields for LG010_3_ej_akuta
No RBX for LG010_3


0    None
dtype: object

# Merge Akuta to one

In [9]:
# Merge all Akuta to one
import datetime, arcpy, os
from pathlib import Path
import pandas as pd

#### REMEMBER TO CHANGE THIS ONE TO AVOID OVERWRITING PREVIOUS ONE ####
#akuta_merged_name = "akuta_2023_05_29"

#LGs_info_name = "Ledningar_Kantträd_2023-05-29.txt"

#prj_dir = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29")
# UPPDRAGSMAPP
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
# LOKALT
#gdb = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\Kantträd_SKB_2023-05-29.gdb")
akuta_location = os.path.join(gdb, 'SKB_XYZ')
akuta_merged_location = os.path.join(gdb, 'akuta')
akuta_merged_path = os.path.join(akuta_merged_location, akuta_merged_name)
#LGs_info_path = prj_dir / LGs_info_name

# LÄGG TILL ATT OM INTE "akut"-DATASET FINNS SÅ SKA DEN SKAPAS

def to_akuta_list(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    print(f"Adding {LG}_{line} to list")
    
    akuta_path = os.path.join(akuta_location, f"{LG}_{line}_akuta")

    akuta_list.append(akuta_path)
    
    print(f"Done adding {LG}_{line} to list")

    
akuta_list = []

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(to_akuta_list, axis=1)

print("Doing merge")
arcpy.Merge_management(akuta_list, akuta_merged_path)


Adding LG010_3 to list
Done adding LG010_3 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\RBX och SKB\\Kantträd - 2023-05-29-test\\Kantträd_SKB_2023-05-29.gdb\\akuta\\akuta_2023_05_29'>

# Merge ej_akuta to one

In [10]:
# Merge all ej_akuta to one
import datetime, arcpy, os
from pathlib import Path
import pandas as pd

#### REMEMBER TO CHANGE THIS ONE TO AVOID OVERWRITING PREVIOUS ONE ####
#ej_akuta_merged_name = "ej_akuta_2023_05_29"

#LGs_info_name = "Ledningar_Kantträd_2023-05-29.txt"

#prj_dir = Path(r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29")
# UPPDRAGSMAPP
#gdb = prj_dir / "SVK_2021_GIS" / "SVK_2021_GIS.gdb"
# LOKALT
#gdb = Path(r"C:\Users\semnve\Desktop\Kantträd 2023-02-04\SVK_SKB_2023-02-06.gdb")
gdb = os.path.join(prj_dir, "Kantträd_SKB_2023-05-29.gdb")
ej_akuta_location = os.path.join(gdb, 'SKB_XYZ')
ej_akuta_merged_location = os.path.join(gdb, 'ej_akuta')
ej_akuta_merged_path = os.path.join(ej_akuta_merged_location, ej_akuta_merged_name)
#LGs_info_path = prj_dir / LGs_info_name

def to_ej_akuta_list(powerline):
    LG = powerline[0]
    line = powerline[1]
    
    print(f"Adding {LG}_{line} to list")
    
    ej_akuta_path = os.path.join(ej_akuta_location, f"{LG}_{line}_ej_akuta")

    ej_akuta_list.append(ej_akuta_path)
    
    print(f"Done adding {LG}_{line} to list")

    
ej_akuta_list = []

LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(to_ej_akuta_list, axis=1)

print("Doing merge")
arcpy.Merge_management(ej_akuta_list, ej_akuta_merged_path)

#print(ej_akuta_list)
#print(ej_akuta_merged_path)

Adding LG010_3 to list
Done adding LG010_3 to list
Doing merge


<Result 'C:\\Users\\semnve\\Desktop\\RBX och SKB\\Kantträd - 2023-05-29-test\\Kantträd_SKB_2023-05-29.gdb\\ej_akuta\\ej_akuta_2023_05_29'>

# Klipp träffpunkter

In [11]:
import arcpy
import os
import pandas as pd


#version = "2023_05_29"

#gdb = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29\Kantträd_SKB_2023-05-29.gdb"
#LGs_info_name = "Ledningar_Kantträd_2023-05-29.txt"
fc_ledningsgator = os.path.join(gdb, "Gator2021")
fc_stationsomraden = os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

#fd_akuta = os.path.join(gdb, "akuta")
fd_ej_akuta = os.path.join(gdb, "ej_akuta")
fd_traffpunkter = os.path.join(gdb, "traffpunkter")

#fc_akuta = os.path.join(fd_akuta, f"akuta_{version}")
fc_ej_akuta = os.path.join(fd_ej_akuta, f"ej_akuta_{version}")
fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(gdb, "tmp_layer_traffpunkter")
#tmp_layer_ledningsgator = os.path.join(gdb, "tmp_layer_ledningsgator")
#tmp_layer_stationsomraden = os.path.join(gdb, "tmp_layer_stationsomraden")

# LÄGG TILL I SKRIPT ATT DET KIKAR OM DET FINNS ETT DATASET SOM HETER "traffpunkter" ANNARS SKAPA DET

# Gör temporära lager att arbeta med
# Ta för säkerhets skull bort eventuella existerande lager
arcpy.management.Delete(tmp_layer_traffpunkter)
arcpy.management.MakeFeatureLayer(fc_ej_akuta, tmp_layer_traffpunkter)
#arcpy.management.Delete(tmp_layer_ledningsgator)
#arcpy.management.MakeFeatureLayer(fc_ledningsgator, tmp_layer_ledningsgator)
#arcpy.management.Delete(tmp_layer_stationsomraden)
#arcpy.management.MakeFeatureLayer(fc_ej_akuta, tmp_layer_stationsomraden)
#
# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_ledningsgator,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = fc_stationsomraden,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, behåll punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Kopiera ej_akuta till traffpunkter
arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter)

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)


<Result 'true'>

# Rensning onödig information

In [13]:
# Rensar rådata från onödig information
# Detta gäller bara dataset SKB_XYZ


import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

#LGs_info_file = "Ledningar_Kantträd_2023-05-29.txt"
#prj_dir = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29"
SKB_XYZ_fd = os.path.join(prj_dir, gdb, "SKB_XYZ")                                                                                            

#LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    fieldnames = ["_SKB_XYZ", "_akuta", "_ej_akuta"]
    
    for fieldname in fieldnames:
        fc_SKB = f"{LG}_{line}{fieldname}"
        SKB_path = os.path.join(SKB_XYZ_fd, fc_SKB)
        # Activating workspace
        arcpy.env.workspace = SKB_path
        arcpy.env.overwriteOutput = True
        
        if arcpy.Exists(fc_SKB):
            SKB_XYZ_fc = f"{LG}_{line}{fieldname}"
            print(f"Altering fields for {LG}_{line}{fieldname}")
            path_joined = os.path.join(SKB_XYZ_fd, f'{LG}_{line}{fieldname}')
            field_names = [f.name for f in arcpy.ListFields(path_joined)]

            if "GIS_ID" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "GIS_ID")
            if "MAX_TILLVAXT" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "MAX_TILLVAXT")
            if "created_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_user")
            if "created_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "created_date")
            if "last_edited_user" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_user")
            if "last_edited_date" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "last_edited_date")
            if "ATGARDSAR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "ATGARDSAR")
            if "LEVERANTOR" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTOR")
            if "LEVERANTORS_DATUM" in field_names:    
                arcpy.management.DeleteField(
                    path_joined,
                    "LEVERANTORS_DATUM")
    else:
        print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

Altering fields for LG010_3_SKB_XYZ
Altering fields for LG010_3_akuta
Altering fields for LG010_3_ej_akuta
No RBX for LG010_3


0    None
dtype: object

In [14]:
# Rensar rådata från onödig information
# Detta gäller bara dataset "akuta", "ej_akuta", "traffpunkter"


import arcpy, os, glob, shutil
from pathlib import Path
import pandas as pd

#LGs_info_file = "Ledningar_Kantträd_2023-05-29.txt"
#prj_dir = r"C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29"
#SKB_XYZ_fd = os.path.join(prj_dir,"SVK_2021_GIS","SVK_2021_GIS.gdb","SKB_XYZ")                                                                                            

#LGs_info_path = os.path.join(prj_dir, LGs_info_file)



def alterfield(powerline):    
    LG = powerline[0]
    line = powerline[1]
    #SKB_XYZ_fc = f"{LG}_{line}_SKB_XYZ"
    
    
    datasetnames = ["akuta", "ej_akuta", "traffpunkter"]
    fieldnames = ["2023_05_29"]
    
    for dataset in datasetnames:
        print(dataset)
        for fieldname in fieldnames:
            fc_SKB = f"{dataset}_{fieldname}"
            if "akuta" in dataset:
                SKB_location = os.path.join(prj_dir,"Kantträd_SKB_2023-05-29.gdb",f"{dataset}")
            else:
                SKB_location = os.path.join(prj_dir,"Kantträd_SKB_2023-05-29.gdb",f"{dataset}")
            print(SKB_location)
            SKB_path = os.path.join(SKB_location, fc_SKB)
 
            # Activating workspace
            arcpy.env.workspace = SKB_path
            arcpy.env.overwriteOutput = True

            if arcpy.Exists(fc_SKB):
                SKB_XYZ_fc = fc_SKB
                print(f"Altering fields for {dataset}_{fieldname}")
                path_joined = os.path.join(SKB_location, f'{dataset}_{fieldname}')
                field_names = [f.name for f in arcpy.ListFields(SKB_path)]

                if "GIS_ID" in field_names:    
                        arcpy.management.DeleteField(
                            path_joined,
                            "GIS_ID")
                if "MAX_TILLVAXT" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "MAX_TILLVAXT")
                if "created_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_user")
                if "created_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "created_date")
                if "last_edited_user" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_user")
                if "last_edited_date" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "last_edited_date")
                if "ATGARDSAR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "ATGARDSAR")
                if "LEVERANTOR" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTOR")
                if "LEVERANTORS_DATUM" in field_names:    
                    arcpy.management.DeleteField(
                        path_joined,
                        "LEVERANTORS_DATUM")
                
                dataset = SKB_location
                sr = arcpy.SpatialReference("SWEREF99_TM")
                arcpy.DefineProjection_management(dataset, sr)
                # Assigning domains to fields
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
                arcpy.management.AssignDomainToField(
                    fc_SKB, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
    #else:
    #    print(f"No RBX for {LG}_{line}")
    
LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
LGs_info.apply(alterfield, axis=1)

akuta
C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29-test\Kantträd_SKB_2023-05-29.gdb\akuta
Altering fields for akuta_2023_05_29
ej_akuta
C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29-test\Kantträd_SKB_2023-05-29.gdb\ej_akuta
Altering fields for ej_akuta_2023_05_29
traffpunkter
C:\Users\semnve\Desktop\RBX och SKB\Kantträd - 2023-05-29-test\Kantträd_SKB_2023-05-29.gdb\traffpunkter
Altering fields for traffpunkter_2023_05_29


0    None
dtype: object